# PROGRAMANDO UN MLP: Predicción de cáncer


In [285]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import random
from sklearn.model_selection import train_test_split
from keras.layers import Dense
from sklearn.preprocessing import LabelEncoder
from keras.optimizers import Adam

>**Importamos el csv:**

In [290]:
df = pd.read_csv('cancer_dataset.csv')
df = df.drop('id',axis=1)
df.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


>**Al ver un atributo de tipo string con dos posibles entradas unicamente, lo transformamos a variable numerica(1: Benigno, 0: Maligno) y lo pasamos a binario**

El atributo diagnosis es el que tomaremos como objetivo de predicción

In [291]:
encoder = LabelEncoder().fit_transform
df['diagnosis'] = tf.keras.utils.to_categorical(encoder(df['diagnosis']))
df['diagnosis'].value_counts()

1.0    357
0.0    212
Name: diagnosis, dtype: int64

>**Normalización de datos**

A continuación vamos a aplicarle a los datos un proceso de normalización, en este ejemplo probamos con una formula:
- (x - media(x)) / desviacionTipica(x)

Tomando la media y la desviacion tipica mediante torch

In [292]:
for columna in df.columns[1:]:
    tensor = torch.tensor(df[columna].values)
    df[columna] = (df[columna] - float(torch.mean(tensor))) / float(torch.std(tensor))
df.head()


,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,0.0,1.096100,-2.071512,1.268817,0.983510,1.567087,3.280628,2.650542,2.530249,2.215566,...,1.885031,-1.358098,2.301575,1.999478,1.306537,2.614365,2.107672,2.294058,2.748204,1.935312
1,0.0,1.828212,-0.353322,1.684473,1.907030,-0.826235,-0.486643,-0.023825,0.547662,0.001391,...,1.804340,-0.368879,1.533776,1.888827,-0.375282,-0.430066,-0.146620,1.086129,-0.243675,0.280943
2,0.0,1.578499,0.455786,1.565126,1.557513,0.941382,1.052000,1.362280,2.035440,0.938859,...,1.510541,-0.023953,1.346291,1.455004,0.526944,1.081980,0.854222,1.953282,1.151242,0.201214
3,0.0,-0.768233,0.253509,-0.592166,-0.763792,3.280667,3.399917,1.914213,1.450431,2.864862,...,-0.281217,0.133866,-0.249720,-0.549538,3.391291,3.889975,1.987839,2.173873,6.040726,4.930672
4,0.0,1.748758,-1.150804,1.775011,1.824624,0.280125,0.538866,1.369806,1.427237,-0.009552,...,1.297434,-1.465481,1.337363,1.219651,0.220362,-0.313119,0.612640,0.728618,-0.867590,-0.396751


Pero al evaluarla vemos que hay valores negativos y mayores a 1, asique lo normalizamos con otra formula:
- (x - valorMinimo(x)) / (valorMaximo(x) - valorMinimo(x))

In [293]:
for columna in df.columns[1:]:
    df[columna] = (df[columna] - min(df[columna])) / (max(df[columna]) - min(df[columna]))
df.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,0.0,0.521037,0.022658,0.545989,0.363733,0.593753,0.792037,0.703140,0.731113,0.686364,...,0.620776,0.141525,0.668310,0.450698,0.601136,0.619292,0.568610,0.912027,0.598462,0.418864
1,0.0,0.643144,0.272574,0.615783,0.501591,0.289880,0.181768,0.203608,0.348757,0.379798,...,0.606901,0.303571,0.539818,0.435214,0.347553,0.154563,0.192971,0.639175,0.233590,0.222878
2,0.0,0.601496,0.390260,0.595743,0.449417,0.514309,0.431017,0.462512,0.635686,0.509596,...,0.556386,0.360075,0.508442,0.374508,0.483590,0.385375,0.359744,0.835052,0.403706,0.213433
3,0.0,0.210090,0.360839,0.233501,0.102906,0.811321,0.811361,0.565604,0.522863,0.776263,...,0.248310,0.385928,0.241347,0.094008,0.915472,0.814012,0.548642,0.884880,1.000000,0.773711
4,0.0,0.629893,0.156578,0.630986,0.489290,0.430351,0.347893,0.463918,0.518390,0.378283,...,0.519744,0.123934,0.506948,0.341575,0.437364,0.172415,0.319489,0.558419,0.157500,0.142595


>**Separamos csv de dos dataset, uno de entrenamiento y otro de predicción**

In [300]:
x_train,x_test,y_train,y_test = train_test_split(
    df.iloc[:,1:],
    df.iloc[:,0],
    test_size= 0.25,
    shuffle= True, 
    random_state= random.randint(0,42)
    )

print(f'TRAIN:\n{x_train.shape}\n{y_train.shape}')
print(f'\nTEST:\n{x_test.shape}\n{y_test.shape}')


TRAIN:
(426, 30)
(426,)

TEST:
(143, 30)
(143,)


>**Creamos el modelo de MLP y lo entrenamos**

In [295]:
model = tf.keras.models.Sequential(
    [
        Dense(units = 30, activation= 'relu'),
        Dense(units = 15, activation= 'relu'),
    ]
)
model.compile(optimizer=Adam(learning_rate=0.33), loss='mean_squared_error', metrics=['accuracy'])
model.fit(x_train,y_train,epochs=100)

Epoch 1/100
14/14 [==============================] - 0s 680us/step - loss: 8.7594 - accuracy: 0.2958
Epoch 2/100
14/14 [==============================] - 0s 478us/step - loss: 0.6197 - accuracy: 0.3803
Epoch 3/100
14/14 [==============================] - 0s 503us/step - loss: 0.6197 - accuracy: 0.3803
Epoch 4/100
14/14 [==============================] - 0s 478us/step - loss: 0.6197 - accuracy: 0.3803
Epoch 5/100
14/14 [==============================] - 0s 899us/step - loss: 0.6197 - accuracy: 0.3803
Epoch 6/100
14/14 [==============================] - 0s 476us/step - loss: 0.6197 - accuracy: 0.3803
Epoch 7/100
14/14 [==============================] - 0s 1ms/step - loss: 0.6197 - accuracy: 0.3803
Epoch 8/100
14/14 [==============================] - 0s 2ms/step - loss: 0.6197 - accuracy: 0.3803
Epoch 9/100
14/14 [==============================] - 0s 1ms/step - loss: 0.6197 - accuracy: 0.3803
Epoch 10/100
14/14 [==============================] - 0s 536us/step - loss: 0.6197 - accuracy: 0.

>**Realizamos la predicción sobre el dataset de predicciones**

In [301]:
y_pred = model.predict(x_test)
y_pred = pd.DataFrame(y_pred)
y_pred.head()



5/5 [==============================] - 0s 569us/step


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [302]:
y_pred.value_counts()

0    1    2    3    4    5    6    7    8    9    10   11   12   13   14 
0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0    143
dtype: int64